# Лабораторная работа 1. Pandas, метод ближайших соседей и решающие деревья.

ФИО: Викулин Всеволод Александрович
 
Группа:  517

In [1]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США за январь-апрель 2008 года.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [3]:
df = pd.read_csv("2008.csv.bz2")

In [4]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [22]:
df_cancelled = df[df["Cancelled"] != 0]
df_cancelled.groupby("CancellationCode").count()["Year"].idxmax()

'B'

In [33]:
mean_d, min_d, max_d = df["Distance"].mean(), df["Distance"].min(), df["Distance"].max()
print mean_d, min_d, max_d
strange_flights = df[df["Distance"] == min_d]
print strange_flights[["FlightNum", "Year", "Month", "DayofMonth", "DayOfWeek", "Origin", "Dest"]]
print "mean distance for first strange flight " + str(df[df["FlightNum"] == strange_flights["FlightNum"].\
                                                       iloc[0]]["Distance"].mean())
print "mean distance for second strange flight " + str(df[df["FlightNum"] == strange_flights["FlightNum"].\
                                                        iloc[1]]["Distance"].mean())
print df[df["FlightNum"] == strange_flights["FlightNum"].iloc[0]]["Distance"].value_counts()
print df[df["FlightNum"] == strange_flights["FlightNum"].iloc[1]]["Distance"].value_counts()

726.387029425 11 4962
         FlightNum  Year  Month  DayofMonth  DayOfWeek Origin Dest
2547298       4988  2008      5          15          4    JFK  LGA
4392215       5572  2008      8          10          7    JFK  LGA
mean distance for first strange flight 770.208888889
mean distance for second strange flight 384.751815981
834    130
903     59
321     18
223     11
712      4
414      1
96       1
11       1
Name: Distance, dtype: int64
329    507
544    243
134     42
363     27
508      4
669      2
11       1
Name: Distance, dtype: int64


Нет, минимальное расстоние в 11 км не выглядит удивительным, так как если Вы посмотрите на Origin и Dest, (IATA code начального и конечного аэропортов) то в  обоих случаях это будут два аэропорта из города Нью-Йорк. Это как из Домодедово во Внуково, только ближе 

In [20]:
df.groupby("Origin").count()["Year"].idxmax()

'ATL'

Яндекс сказал, что такой IATA code у Международного аэропорта Хартсфилд-Джексон в Атланте

In [3]:
df.groupby("Origin").mean()["AirTime"].idxmax()

'SJU'

Спросим у Яндекса. Международный аэропорт имени Луиса Муньоса Маринааэропорт совместного базирования, расположенный в городе Каролина, в пяти километрах к юго-востоку от столицы Пуэрто-Рико, города Сан-Хуан. Является крупнейшим коммерческим аэропортом в Пуэрто-Рико. Но Пуэрто-Рико вроде на острове находится, так что дейтвительно летать всегда далеко.

In [22]:
df_task_6 = df.groupby("Origin").filter(lambda x: len(x) > 1000)
def true_delay_percent(x):
    return float(len(x[x["DepDelay"] > 0]))/float(len(x))
df_task_6.groupby("Origin").agg(true_delay_percent)["Year"].idxmax()

'DAL'

Это аэропорт города Даллас, штат Техас. Если посмотреть на значение, то более половины рейсов у него задержаны. Шшшикарно.

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [5]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

/home/vsevolod/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [7]:
"""All this  functions need for calculation second and third metric in out task.
Firste simple metric works without this funcs."""
def create_one_count(x, value_counts):
    if x not in value_counts:
        return 0, 0
    new = value_counts[x]
    return new, value_counts[value_counts <= new].sum()

def create_counts(column, train_value_counts=None):
    if train_value_counts is None:
        value_counts = column.value_counts()
        return np.array(map(lambda x : create_one_count(x=x, value_counts=value_counts), column)), value_counts
    return np.array(map(lambda x : create_one_count(x=x, value_counts=train_value_counts), column))

def make_train_counters(train_data):
    second_metric_freq = []
    final_train_counters = {}
    all_names = list(train_data.columns.values)
    for column in all_names:
        temp, new_value_counts = create_counts(train_data[column])
        train_data[str(column) + " counts"] = temp[:,0]
        second_metric_freq.append(list(temp[:,1]))
        final_train_counters[column] = new_value_counts
        del train_data[column]
    return train_data.values, final_train_counters, np.array(second_metric_freq).transpose()

def make_test_counters(test_data, train_counters):
    all_names = list(test_data.columns.values)
    second_metric_freq = []
    for column in all_names:
        new_value_counts = train_counters[column]
        temp = create_counts(test_data[column], new_value_counts)
        test_data[str(column) + " counts"] = temp[:,0]
        second_metric_freq.append(list(temp[:,1]))
        del test_data[column]
    return test_data.values, np.array(second_metric_freq).transpose()

In [8]:
class CustomKNN():
    """my own simple KNN realization.
    SKLEARN-like syntax Enjoy!"""
    metric = None
    n_neighbors = None
    X_train = None
    y_train = None
    
    def __init__(self):
        """Init our KNN
        we have some parametrs in
        fit and predict methods"""
        
    def fit(self, X_train, y_train, metric):
        """Simple fit function.
        I like lazzy-learn."""
        self.metric = metric
        self.X_train = X_train
        self.y_train = y_train
        if self.metric != "first":      
            self.freq_train, self.counter, second_train = make_train_counters(pd.DataFrame(X_train))
        if self.metric == "second":
            self.freq_train = second_train /float(self.X_train.shape[0])
                   
    def predict(self, X_test, batch_size=1000, n_neighbors=10):
        """Let s try to realize 
        searching k nneighbors"""
        self.n_neighbors = n_neighbors
        all_predicts = np.ones(X_test.shape[0])
        old_i = 0
        if self.metric != "first":
            freq_test, second_test = make_test_counters(pd.DataFrame(X_test),  self.counter)
        else:
            freq_test = np.zeros((X_test.shape[0],1))
        if self.metric == "second":
            freq_test = second_test/float(self.X_train.shape[0])
        for i in range (0, len(X_test) / batch_size + 1*(len(X_test) % batch_size != 0)):
            batch_predict = self.predict_for_one_batch(X_test[i*batch_size:(i+1)*batch_size,:],\
                                                    metric=self.metric,\
                                                    freq_test=freq_test[i*batch_size:(i+1)*batch_size,:])
            all_predicts[old_i : old_i + len(batch_predict)] = batch_predict
            old_i = len(batch_predict)*(i+1)
        return all_predicts
    
    def predict_for_one_batch(self, batch, metric="first", freq_test=None):
        if metric == "first":
            dist_matrix = (np.sum((self.X_train[:, np.newaxis] != batch[np.newaxis, :]) , axis=2))
            
        if metric == "second":
            not_yet_d_mat = (self.X_train[:, np.newaxis] != batch[np.newaxis, :]) + \
                            ((self.X_train[:, np.newaxis] == batch[np.newaxis, :]) *\
                             (self.freq_train[:, np.newaxis] * freq_test[np.newaxis, :]))
            dist_matrix = (np.sum(not_yet_d_mat , axis=2))
            
        if metric == "third":
            dist_matrix = np.sum( ((self.X_train[:, np.newaxis] != batch[np.newaxis, :]) *\
                                   (np.log(self.freq_train[:, np.newaxis] + 1) * np.log(freq_test[np.newaxis, :] + 1)) ) , axis=2)
        
        dist_matrix = np.argsort(dist_matrix, axis=0)[0:self.n_neighbors].transpose()
        return [self.y_train[i].mean() for i in dist_matrix]

In [52]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
test_knn = CustomKNN()
test_knn.fit(X_train, y_train, metric='first')
import time
t0 = time.time()
result = test_knn.predict(X_test, batch_size=1000)
print time.time() - t0

11.0521669388


Я применил всю свою интеллектуальную мощь, чтобы эта штука считалась максимально быстро. С размером батча 1000, первая метрика считается на тесте всего 10 - 11 секунд. Если увеличивать размер батча - будет еще быстрее, но мне жалко оперативы. Две другие метрики работают уже не так быстро, но это не потому что  я их плохо написал, (то есть, не только поэтому) а потому что сначала нам для второй метрики надо посчитать дурацкую матрицу из сумм вероятностей. Считается она довольно долго в тех 4-х функциях, которые выше. Сами вычисления же, если  у нас будет эта матрица в руках (можно просто ее заранее посчитать, но тут такой трюк не показан, так как это костыль, но само исследование проводилось с ним, чтобы не ждать лишнюю тонну секунд), делаются не сильно дольше чем первая метрика. Я горжусь этой скоростью!

In [53]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, result)

0.83088009598014845

In [31]:
from sklearn.metrics import roc_auc_score
test_knn.fit(X_train, y_train, metric='second')
result = test_knn.predict(X_test, batch_size=1000)
roc_auc_score(y_test, result)

0.81983019449781858

In [15]:
from sklearn.metrics import roc_auc_score
test_knn.fit(X_train, y_train, metric='third')
result = test_knn.predict(X_test, batch_size=1000)
roc_auc_score(y_test, result)

0.82096677787194638

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

Здесь   я формил некоторую вооображемую табличку, для каждой из метрик указано качество для всех провеернных количествах соседей. Все делал тупо, перебирал в цикле все эти варианты и  для разных random state сплитов, чтобы было честнее, на самом деле АУК сильно зависит от того, как Вы поделите выборку, а затем записывал, но я старался!!!

первая : 20 - 0.824, 19 - 0.824, 18-0.823, 17 - 0.826, 16 - 0.828, 15 - 0.828, 14 - 0.831, 13 - 0.831, 12-0.829, 11 -  0.831,  10 - 0.831, 9 - 0.830, 8 - 0.830, 7 - 0.829, 6 - 0.827


вторая : 20 -0.811, 19 - 0.814, 18 -0.815, 17 - 0.819, 16 - 0.822, 15 - 0.821,v14- 0.819,13 - 0.821, 12 - 0.821, 11- 0.82, 10 - 0.82, 9 - 0.821, 8 - 0.822, 7 - 0.823, 6 - 0.817


третья : 20- 0.807, 19 - 0.808, 18 - 0.809, 17 - 0.815, 16 -0.810, 15- 0.812,14 -0.817 ,13 - 0.818, 12 - 0.82, 11 - 0.819 , 10 - 0.821, 9 - 0.8211, 8 -  0.8197, 7 - 0.8197, 6 - 0.815

Краткое резюме по поводу метрик. Тестирование было не очень честное, потому что я старался как-то усреднять по нескольким случайным разбиениям, оказалось, что скор сильно будет разный от этого. Возможно, я просто фигово реализовал 2 и 3 метрики, потому что чисто интуитивно мне кажется, что они должны работать лучше. Но ошибку я найти не смог, так что оставляю такие результаты, которые немножко зависят от рандома. Ну зато, хотя бы быстро :(

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [174]:
def create_one_count(x, value_counts):
    if x not in value_counts:
        return 0
    return value_counts[x]

def create_counts(column, train_value_counts=None):
    if train_value_counts is None:
        value_counts = column.value_counts()
        return map(lambda x : create_one_count(x=x, value_counts=value_counts), column), value_counts
    return map(lambda x : create_one_count(x=x, value_counts=train_value_counts), column)

def make_train_counters(train_data, train_target):
    final_train_counters = {}
    all_names = list(train_data.columns.values)
    for column in all_names:
        train_data[column + " counts"], new_value_counts = create_counts(train_data[column])
        success_train_data = train_data[train_target == 1]
        temp, new_value_successes = create_counts(success_train_data[column])
        train_data[column + " successes"] = create_counts(train_data[column], new_value_successes)
        train_data[column + " smooth"] = (train_data[column + " successes"] + 1.0).values/(train_data[column + " counts"] + 2.0).values
        final_train_counters[column] = (new_value_counts, new_value_successes)
        del train_data[column]
    return train_data, final_train_counters



def make_test_counters(test_data, train_counters, num_folds=1):
    all_names = list(test_data.columns.values)
    alpha = 1.0
    if num_folds != 1:
        alpha =  float(num_folds - 1)/(float(num_folds))
    for column in all_names:
        new_value_counts = train_counters[column][0]
        new_value_successes = train_counters[column][1]
        test_data[column + " counts"] = create_counts(test_data[column], new_value_counts)
        test_data[column + " successes"] = create_counts(test_data[column], new_value_successes)
        test_data[column + " counts"] *= alpha
        test_data[column + " counts"] = test_data[column + " counts"].round()
        test_data[column + " successes"] *= alpha
        test_data[column + " successes"] = test_data[column + " successes"].round()
        test_data[column + " smooth"] = (test_data[column + " successes"] + 1.0).values/(test_data[column + " counts"] + 2.0).values
        del test_data[column]
    return test_data

def simple_realization(X_train, y_train, X_test):
    X_train, train_counters = make_train_counters(X_train, y_train)
    X_test = make_test_counters(X_test, train_counters)
    return X_train, X_test

from sklearn.model_selection import KFold
def hard_realization(X_train, y_train, X_test, num_folds=5):
    kf = KFold(num_folds)
    new_train_data = pd.DataFrame()
    new_train_target = pd.Series()
    for train, test in kf.split(X_train):
        kf_train = X_train.iloc[train].copy()
        kf_test = X_train.iloc[test].copy()
        kf_y_train = y_train.iloc[train]
        kf_y_test = y_train.iloc[test]
        kf_train, kf_train_counters = make_train_counters(kf_train, kf_y_train)
        kf_test = make_test_counters(kf_test, kf_train_counters)
        new_train_data = new_train_data.append(kf_test.copy())
        new_train_target = new_train_target.append(kf_y_test)
    all_x_train, train_counters = make_train_counters(X_train, y_train)
    new_X_test = make_test_counters(X_test.copy(), train_counters, num_folds=num_folds)
    new_X_train = new_train_data
    new_y_train = new_train_target
    return new_X_train, new_y_train, new_X_test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train1, X_test1 =  simple_realization(X_train, y_train, X_test)
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train2, y_train2, X_test2 = hard_realization(X_train, y_train, X_test)

In [179]:
X_test2.head()

,RESOURCE counts,RESOURCE successes,RESOURCE smooth,MGR_ID counts,MGR_ID successes,MGR_ID smooth,ROLE_ROLLUP_1 counts,ROLE_ROLLUP_1 successes,ROLE_ROLLUP_1 smooth,ROLE_ROLLUP_2 counts,...,ROLE_TITLE smooth,ROLE_FAMILY_DESC counts,ROLE_FAMILY_DESC successes,ROLE_FAMILY_DESC smooth,ROLE_FAMILY counts,ROLE_FAMILY successes,ROLE_FAMILY smooth,ROLE_CODE counts,ROLE_CODE successes,ROLE_CODE smooth
24312,3.0,3.0,0.800000,26.0,26.0,0.964286,11957.0,11379.0,0.951585,1398.0,...,0.940223,74.0,72.0,0.960526,6132.0,5798.0,0.945386,985.0,927.0,0.940223
12246,4.0,4.0,0.833333,22.0,22.0,0.958333,11957.0,11379.0,0.951585,522.0,...,0.974576,22.0,22.0,0.958333,6132.0,5798.0,0.945386,116.0,114.0,0.974576
15099,10.0,9.0,0.833333,13.0,13.0,0.933333,11957.0,11379.0,0.951585,2500.0,...,0.883838,50.0,50.0,0.980769,734.0,695.0,0.945652,196.0,174.0,0.883838
22149,0.0,0.0,0.500000,26.0,25.0,0.928571,11957.0,11379.0,0.951585,941.0,...,0.921374,3866.0,3614.0,0.934592,6132.0,5798.0,0.945386,2618.0,2413.0,0.921374
24776,11.0,10.0,0.846154,4.0,2.0,0.500000,34.0,26.0,0.750000,34.0,...,0.881944,311.0,277.0,0.888179,1477.0,1330.0,0.899932,718.0,634.0,0.881944


In [180]:
X_test1.head()

,RESOURCE counts,RESOURCE successes,RESOURCE smooth,MGR_ID counts,MGR_ID successes,MGR_ID smooth,ROLE_ROLLUP_1 counts,ROLE_ROLLUP_1 successes,ROLE_ROLLUP_1 smooth,ROLE_ROLLUP_2 counts,...,ROLE_TITLE smooth,ROLE_FAMILY_DESC counts,ROLE_FAMILY_DESC successes,ROLE_FAMILY_DESC smooth,ROLE_FAMILY counts,ROLE_FAMILY successes,ROLE_FAMILY smooth,ROLE_CODE counts,ROLE_CODE successes,ROLE_CODE smooth
24312,4.0,4.0,0.833333,33.0,33.0,0.971429,14946.0,14224.0,0.951632,1748.0,...,0.940795,92.0,90.0,0.968085,7665.0,7248.0,0.945481,1231.0,1159.0,0.940795
12246,5.0,5.0,0.857143,27.0,27.0,0.965517,14946.0,14224.0,0.951632,652.0,...,0.972789,27.0,27.0,0.965517,7665.0,7248.0,0.945481,145.0,142.0,0.972789
15099,12.0,11.0,0.857143,16.0,16.0,0.944444,14946.0,14224.0,0.951632,3125.0,...,0.882591,63.0,63.0,0.984615,917.0,869.0,0.946681,245.0,217.0,0.882591
22149,0.0,0.0,0.500000,32.0,31.0,0.941176,14946.0,14224.0,0.951632,1176.0,...,0.921221,4833.0,4518.0,0.934643,7665.0,7248.0,0.945481,3273.0,3016.0,0.921221
24776,14.0,12.0,0.812500,5.0,3.0,0.571429,43.0,33.0,0.755556,43.0,...,0.881111,389.0,346.0,0.887468,1846.0,1662.0,0.899892,898.0,792.0,0.881111


In [204]:
clf =  KNeighborsClassifier(n_neighbors=40, metric='hamming', weights='distance')
clf.fit(X_train1, y_train)
from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, clf.predict_proba(X_test1)[:,1])

0.819666567466


In [232]:
clf =  KNeighborsClassifier(n_neighbors=40, metric='hamming', weights='distance')
clf.fit(X_train2, y_train2)
print roc_auc_score(y_test, clf.predict_proba(X_test2)[:,1])

0.751087791788


#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [58]:
def create_new_columns(data):
    new_data = data.copy()
    all_columns = list(data.columns.values)
    for i in range(0, len(all_columns)):
        for j in range(i+1, len(all_columns)):
            new_data[all_columns[i] + "&" + all_columns[j]] = (new_data[all_columns[i]].map(str) \
                                          +  "&" + new_data[all_columns[j]].map(str))
    return new_data

In [235]:

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train = create_new_columns(X_train)
X_test = create_new_columns(X_test)
X_train1, X_test1 =  simple_realization(X_train, y_train, X_test)


X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train = create_new_columns(X_train)
X_test = create_new_columns(X_test)
X_train2, y_train2, X_test2 = hard_realization(X_train, y_train, X_test)

In [244]:
clf =  KNeighborsClassifier(n_neighbors=40, metric='hamming', weights='distance')
clf.fit(X_train1, y_train)
from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, clf.predict_proba(X_test1)[:,1])

0.834672181643


In [250]:
clf =  KNeighborsClassifier(n_neighbors=40, metric='hamming', weights='distance')
clf.fit(X_train2, y_train2)
print roc_auc_score(y_test, clf.predict_proba(X_test2)[:,1])

0.81437712823


## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [59]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
from sklearn.tree import DecisionTreeClassifier
X_train = create_new_columns(X_train)
X_test = create_new_columns(X_test)
X_train, X_test =  simple_realization(X_train, y_train, X_test)

In [166]:
clf =  DecisionTreeClassifier(max_depth=11, min_samples_leaf=5)
clf.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

0.687284211798


#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

In [172]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=700)
clf.fit(X_train, y_train)
print roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

0.724581470627


#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [183]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train = create_new_columns(X_train)
X_test = create_new_columns(X_test)
X_train, y_train, X_test =  hard_realization(X_train, y_train, X_test)

In [180]:
clf =  DecisionTreeClassifier(max_depth=11, min_samples_leaf=5)
clf.fit(X_train, y_train)
from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

0.72083593016


In [179]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=700)
clf.fit(X_train, y_train)
print roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

0.867365599398


Честно сказать, я был не сильно удивлен, что скор при валидационном построении признаков сильно лучше, чем при обычном, просто по той причине, что у нас нету дата лика. В простом варианте, у нас в признаках неявно сидит сама целевая функция. Единственное, что смущало, это то, как я подобрал параметр альфа, которым мы будем занижать счетчики для тестовой выборки. Этим я объснял более плохой скор для KNN с валидационными признаками. Но в том случае, видимо при выборе KNN и метрики хэмминга это вообще не помогает, а деревья сильно переобучаются (такие по своей природе). Кстати, на мой взгляд, в случае построении парных признаков дата лик должен даже усиливаться, так как мы строим новый признак по двум другим, которые уже заведома зависят от целевой переменной. Может быть, этим можно объснить, что при парных признаках "сложный" KNN подобрался к "простому". 

Здесь вы можете поделиться своими мыслями о задании.

Кратко по поводу заданий. Первая часть - очень простая, но проблема в том, что групбай ест нерелаьное количество оперативы, так что если памяти < 8 гб, то на таком ноуте домашку делать нереально, датасет очень большой. Хотелось бы поменьше.
Вторая часть довольно захватывающая, но на реализацию быстрого KNN ушло много времени, за которое я бы мог посомтреть сериал и поэтому быть более готовым к вашим КР. Реально, много времени ушло :(
Третья часть довольно простая. С ней проблем не было, фром склерн импорт дерево и все. Но в ней правда и не было ничего захватывающего.
Резюмируя, суммарная сложность терпимая, суммарное удовлетворение от задачи выше ожидаемого.

А здесь вставьте смешную картинку.

<img src="risovach.ru.jpg">

ну и немножко старых добрых баянов (прошлая несмешная картинка была моей!)

<img src="machineLearning.png">

А здесь посоветуйте преподавателям хороший фильм или сериал.

Не сериал, но видео годное: https://www.youtube.com/watch?v=PI7SLOovO5c